# Работа с Datastores и Datasets в Azure ML

__Цель лабораторной работы:__

- создание и управление Хранилищами данных (Datastore)
- регистрация и управление Наборами данных (Datasets)

## Подготовка среды

Импорт необходимых модулей и проверка версии Azure ML SDK:

In [ ]:
import os

import azureml.core
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.datastore import Datastore
from azureml.data.data_reference import DataReference
from msrest.exceptions import HttpOperationError

# Check core SDK version number
print(f'SDK version: {azureml.core.VERSION}')

Получим конфигурацию эксперимента:

In [ ]:
%run core.py

config = get_experiment_config('lab_2B')
config

## Соединение со Azure ML Workspace

Устанавливаем соединение с Рабочей областью в Azure ML:

In [ ]:
ws = Workspace.from_config()
print(f'Successfully connected to Workspace: {ws.name}.')

## Работа с Datastore

## Просмотр существующих Datastore

Получим список зарегистрированных Хранилищ данных, в т.ч. и Хранилища данных по умолчанию:

In [ ]:
# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name)

print('\n')

# Get the default datastore
default_ds = ws.get_default_datastore()
print(f'Default Datastore: {default_ds.name}')

## Создаем новый Datastore

Установка имени нового Datastore и указание информации об Azure Storage Account, где будет распологаться новый Datastore:

In [ ]:
datastore_name = config['core']['datastore_name']

# Azure Storage Account Info
storage_account_name = config['storage_account_name']
storage_container_name = config['storage_container_name']
storage_account_key = config['storage_account_key'] # WARN: insert your storage account key here

In [ ]:
print(datastore_name, storage_account_name, storage_container_name)

Создаем Datastore, если он уже не существует:

In [ ]:
try:
    new_datastore = Datastore.get(ws, datastore_name)
    print(f'Blob Datastore with name {new_datastore.name} was found!')

except HttpOperationError:
    new_datastore = Datastore.register_azure_blob_container(
        workspace=ws,
        datastore_name=datastore_name,
        account_name=storage_account_name,
        container_name=storage_container_name,
        account_key=storage_account_key)
    print(f'Registered blob datastore with name {new_datastore.name}')

Получаем информацию о cозданном Datastore:

In [ ]:
print(f'Datastore {new_datastore.name} based on {new_datastore.datastore_type} in storage account named {new_datastore.account_name}')

## Загрузка данных из Datastore

Делаем созданный Datastore хранилищем по умолчанию (для удобства дальнейшей работы):

In [ ]:
ws.set_default_datastore(new_datastore.name)
ds = ws.get_default_datastore()

print(ds.name)

Загрузка данных (внимание: создайте контейтенер, если его до это не существовала):

In [ ]:
ds.upload_files(files=['../data/diabetes_train.csv', '../data/diabetes_test.csv'], # Upload the diabetes csv files in /data
                       target_path='diabetes-data/', # Put it in a folder path in the datastore
                       overwrite=True, # Replace existing files of the same name
                       show_progress=True)

Зарегистрируем загруженные в Datastore данные, как табличный Dataset:

In [ ]:
diabetes_ds = Dataset.Tabular.from_delimited_files(path=(ds, 'diabetes-data/*.csv'))
diabetes_ds

## Зарегистрируем Dataset

In [ ]:
diabetes_db = diabetes_ds.register(workspace = ws,
                                   name = config['core']['dataset_name'],
                                   description = 'Diabetes Disease Database (Winter School 2020)',
                                   create_new_version = True)

Просмотрим список зарегистрированных Наборов данных:

In [ ]:
print('Available datasets:')

for ds in ws.datasets:
    print(f'\t{ds}')

## Просмотр Набора данных

Скачаем зарегистрированный набор данных и выведем 10 строк:

In [ ]:
diabetes_db_from_azure = ws.datasets.get(config['core']['dataset_name'])

diabetes_df = diabetes_db_from_azure.to_pandas_dataframe()
diabetes_df.sample(10)